In [1]:
import pandas as pd
import numpy as np
import requests #api
import json #json file
from datetime import datetime

import matplotlib.pyplot as plt
from scipy.stats import linregress #stastistical analysis

from pathlib import Path #reading csv - connecting to the csv merges mac and windows

## COVID Vaccine Data

In [2]:
# https://data.chhs.ca.gov/dataset/vaccine-progress-dashboard
CA_vax_path = Path("covid19vaccinesbycounty.csv")
CA_vax_df = pd.read_csv(CA_vax_path)
CA_vax_df

,county,administered_date,total_doses,cumulative_total_doses,partially_vaccinated,total_partially_vaccinated,fully_vaccinated,cumulative_fully_vaccinated,at_least_one_dose,cumulative_at_least_one_dose,california_flag,up_to_date_count,cumulative_up_to_date_count
0,Alameda,2020-01-05,0,0,0,0,0,0,0,0,NaN,0,0
1,Alameda,2020-07-27,0,0,0,0,0,0,0,0,NaN,0,0
2,Alameda,2020-07-29,0,1,0,1,0,0,0,1,NaN,0,0
3,Alameda,2020-07-30,0,1,0,1,0,0,0,1,NaN,0,0
4,Alameda,2020-07-31,0,1,0,1,0,0,0,1,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71791,Yuba,2023-09-09,2,110137,0,4194,0,40280,0,44480,California,2,7382
71792,Yuba,2023-09-11,2,110139,0,4194,0,40280,0,44480,California,2,7384
71793,Yuba,2023-09-12,1,110140,1,4195,0,40280,1,44481,California,1,7385
71794,Yuba,2023-09-13,1,110141,1,4196,0,40280,1,44482,California,1,7386


In [3]:
#county, admin date - month and year - group by, find max cumulative of take all of cumulative columns
cleaned_vaccine = CA_vax_df[['county','administered_date','cumulative_total_doses','cumulative_fully_vaccinated',\
                             'cumulative_at_least_one_dose','cumulative_up_to_date_count']]
cleaned_vaccine

,county,administered_date,cumulative_total_doses,cumulative_fully_vaccinated,cumulative_at_least_one_dose,cumulative_up_to_date_count
0,Alameda,2020-01-05,0,0,0,0
1,Alameda,2020-07-27,0,0,0,0
2,Alameda,2020-07-29,1,0,1,0
3,Alameda,2020-07-30,1,0,1,0
4,Alameda,2020-07-31,1,0,1,0
...,...,...,...,...,...,...
71791,Yuba,2023-09-09,110137,40280,44480,7382
71792,Yuba,2023-09-11,110139,40280,44480,7384
71793,Yuba,2023-09-12,110140,40280,44481,7385
71794,Yuba,2023-09-13,110141,40280,44482,7386


In [16]:
cleaned_vaccine['year'] = cleaned_vaccine['administered_date'].dt.year
cleaned_vaccine['month'] = cleaned_vaccine['administered_date'].dt.month
cleaned_vaccine.head()
#defaults turning datetme to timestamp because more secure

C:\Users\nicoo\AppData\Local\Temp\ipykernel_18428\1148678567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_vaccine['year'] = cleaned_vaccine['administered_date'].dt.year
C:\Users\nicoo\AppData\Local\Temp\ipykernel_18428\1148678567.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_vaccine['month'] = cleaned_vaccine['administered_date'].dt.month


,county,administered_date,cumulative_total_doses,cumulative_fully_vaccinated,cumulative_at_least_one_dose,cumulative_up_to_date_count,year,month
0,Alameda,2020-01-05,0,0,0,0,2020,1
1,Alameda,2020-07-27,0,0,0,0,2020,7
2,Alameda,2020-07-29,1,0,1,0,2020,7
3,Alameda,2020-07-30,1,0,1,0,2020,7
4,Alameda,2020-07-31,1,0,1,0,2020,7


In [13]:
cleaned_vaccine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71796 entries, 0 to 71795
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   county                        71796 non-null  object        
 1   administered_date             71796 non-null  datetime64[ns]
 2   cumulative_total_doses        71796 non-null  int64         
 3   cumulative_fully_vaccinated   71796 non-null  int64         
 4   cumulative_at_least_one_dose  71796 non-null  int64         
 5   cumulative_up_to_date_count   71796 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 3.3+ MB


In [14]:
# cleaned_vaccine['administered_date'] = pd.to_datetime(cleaned_vaccine['administered_date'])
# cleaned_vaccine.info()
#not going to change into date time because i will  be using the breaking a string as their own with '-'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71796 entries, 0 to 71795
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   county                        71796 non-null  object        
 1   administered_date             71796 non-null  datetime64[ns]
 2   cumulative_total_doses        71796 non-null  int64         
 3   cumulative_fully_vaccinated   71796 non-null  int64         
 4   cumulative_at_least_one_dose  71796 non-null  int64         
 5   cumulative_up_to_date_count   71796 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 3.3+ MB


C:\Users\nicoo\AppData\Local\Temp\ipykernel_18428\2045081852.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_vaccine['administered_date'] = pd.to_datetime(cleaned_vaccine['administered_date'])


In [ ]:
#line 14 and 15 CA case urv to break up the string so dates are 01-05-2020 to year and month as 2020 and 07
# Get rid of chaning into datetime